In [ ]:
/l/users/santosh.sanjeev/model_soups/runs/test/fge/Cifar10_2/ResNet50/1.00e-05/

In [3]:
from engine import val_step
from utils.utils import souping, greedy_souping
from models import get_model

import torch
from torch.utils.data import DataLoader
from torchvision import transforms
import torchvision

from dataset import RSNADataset, HAM10000Dataset, AptosDataset
from torchvision.transforms.v2 import AutoAugmentPolicy, functional as F, InterpolationMode, Transform
from torchvision.transforms import v2
torchvision.disable_beta_transforms_warning()
from timm.data.transforms import RandomResizedCropAndInterpolation
import matplotlib.pyplot as plt
import numpy as np
import random
import pandas as pd
import argparse

from utils.utils import EarlyStopper, get_dataset
from torch.utils.data.dataset import Subset

import yaml
import json
import time
import os
import wandb
from utils.utils import load_model
import sys


In [21]:
state_dicts, val_results, test_results = [], [], []
loss = torch.nn.CrossEntropyLoss()
train_loader, val_loader, test_loader = get_dataset('Cifar10', './dataset/cifar""./dataset/cifar/cifar1098_idxs.npy', "Minimal", 'ImageNet', 224, 128, 16, 'Classification')

WEIGHT_PATH = '/l/users/santosh.sanjeev/model_soups/runs/test/fge/Cifar10/DeiT-S/'
for idx, folder in enumerate(sorted(os.listdir(WEIGHT_PATH))):
    print(folder)
    if folder.startswith('output'):
        continue
    folder_path = os.path.join(WEIGHT_PATH, folder)
    for idx, model in enumerate(sorted(os.listdir(folder_path))):
        if model.endswith('RESULTS.csv'):
            continue
        model_path = os.path.join(folder_path, model)
        #read config in train_summary.json
        print(model_path)

        checkpoint = torch.load(model_path, map_location='cuda')
        if '.pth' in model_path:
            state_dicts.append(checkpoint['model'])
            model = get_model('DeiT-S', num_classes=10)
            model.load_state_dict(checkpoint['model'])
            model = model.to('cuda')
        else:        
            state_dicts.append(checkpoint['model_state'])
            model = get_model('DeiT-S', num_classes=10)
            model.load_state_dict(checkpoint['model_state'])
            model = model.to('cuda')
        test_loss, test_acc, test_f1, test_recall, test_kappa, test_auc = val_step(model, test_loader, train_loader, loss, device = 'cuda', classification = 'MultiClass')
        val_loss, val_acc, val_f1, val_recall, val_kappa, val_auc = val_step(model, val_loader, train_loader, loss, device = 'cuda', classification = 'MultiClass')

        val_results.append({'Model Name': idx,
                                        'Val Accuracy': val_acc,
                                        'Val F1': val_f1,
                                        'Val Recall': val_recall,
                                        'Val Kappa': val_kappa,
                                        'Val AUC': val_auc,})
        test_results.append({'Model Name': idx,
                                        'Test Accuracy': test_acc,
                                        'Test F1': test_f1,
                                        'Test Recall': test_recall,
                                        'Test Kappa': test_kappa,
                                        'Test AUC': test_auc,})
        

results_test_df = pd.DataFrame(test_results)
results_val_df = pd.DataFrame(val_results)
print(results_val_df)
print(results_test_df)



print("Uniform souping ...")
alphal = [1 / len(state_dicts) for i in range(len(state_dicts))]
model = get_model('DeiT-S', num_classes=10)
uniform_model = souping(model, state_dicts, alphal)
uniform_model.to('cuda')

uniform_test_loss, uniform_test_acc, uniform_test_f1, uniform_test_recall, uniform_test_kappa, uniform_test_auc = val_step(uniform_model, test_loader, train_loader, loss, 'cuda', 'MultiClass')

#greedy

print("Greedy souping ...")
model = get_model('DeiT-S', num_classes=10)

val_results = list(results_val_df['Val Accuracy'])
val_models = list(results_val_df["Model Name"])
greedy_model, best_ingredients = greedy_souping(state_dicts, val_results, 'DeiT-S', 10, val_loader,train_loader, loss, 'cuda' , 'MultiClass', 'Val Accuracy', val_models =val_models)
greedy_model.to('cuda')
print('VAL INGREDIENTS',best_ingredients)
greedy_test_loss, greedy_test_acc, greedy_test_f1, greedy_test_recall, greedy_test_kappa, greedy_test_auc = val_step(greedy_model, test_loader, train_loader, loss, 'cuda', 'MultiClass')
test_results = list(results_test_df['Test Accuracy'])
test_models = list(results_test_df["Model Name"])
greedy_model_test, best_ingredients_test = greedy_souping(state_dicts, test_results, 'DeiT-S', 10, test_loader, train_loader, loss, 'cuda', 'MultiClass', 'Test Accuracy', test_models)
print('TEST INGREDIENTS', best_ingredients_test)
greedy_model_test.to('cuda')
greedy_test_loss_test, greedy_test_acc_test, greedy_test_f1_test, greedy_test_recall_test, greedy_test_kappa_test, greedy_test_auc_test = val_step(greedy_model_test, test_loader, train_loader, loss, 'cuda', 'MultiClass')


['./dataset/cifar', './dataset/cifar/cifar1098_idxs.npy']
Cifar10 45000 5000 10000
1.00e-04
/l/users/santosh.sanjeev/model_soups/runs/test/fge/Cifar10/DeiT-S/1.00e-04/fge-0.pt


100%|██████████| 40/40 [00:24<00:00,  1.65it/s]


/l/users/santosh.sanjeev/model_soups/runs/test/fge/Cifar10/DeiT-S/1.00e-04/fge-1.pt


100%|██████████| 40/40 [00:24<00:00,  1.63it/s]


/l/users/santosh.sanjeev/model_soups/runs/test/fge/Cifar10/DeiT-S/1.00e-04/fge-10.pt


...

/l/users/santosh.sanjeev/model_soups/runs/test/fge/Cifar10/DeiT-S/5.00e-05/fge-10.pt


100%|██████████| 40/40 [00:24<00:00,  1.63it/s]


/l/users/santosh.sanjeev/model_soups/runs/test/fge/Cifar10/DeiT-S/5.00e-05/fge-11.pt


  0%|          | 0/79 [00:04<?, ?it/s]


OutOfMemoryError: CUDA out of memory. Tried to allocate 296.00 MiB (GPU 0; 23.62 GiB total capacity; 20.40 GiB already allocated; 304.12 MiB free; 20.86 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [18]:
state_dicts, val_results, test_results = [], [], []
loss = torch.nn.CrossEntropyLoss()
train_loader, val_loader, test_loader = get_dataset('Cifar10', './dataset/cifar""./dataset/cifar/cifar1098_idxs.npy', "Minimal", 'ImageNet', 224, 128, 16, 'Classification')

WEIGHT_PATH = '/l/users/santosh.sanjeev/model_soups/runs/test/fge/Cifar10_4/DeiT-S/'
for idx, folder in enumerate(sorted(os.listdir(WEIGHT_PATH))):
    print(folder)
    if folder.startswith('output'):
        continue
    folder_path = os.path.join(WEIGHT_PATH, folder)
    for idx, model in enumerate(sorted(os.listdir(folder_path))):
        if model.endswith('RESULTS.csv'):
            continue
        model_path = os.path.join(folder_path, model)
        #read config in train_summary.json
        print(model_path)

        checkpoint = torch.load(model_path, map_location='cuda')
        if '.pth' in model_path:
            state_dicts.append(checkpoint['model'])
            model = get_model('DeiT-S', num_classes=10)
            model.load_state_dict(checkpoint['model'])
            model = model.to('cuda')
        else:        
            state_dicts.append(checkpoint['model_state'])
            model = get_model('DeiT-S', num_classes=10)
            model.load_state_dict(checkpoint['model_state'])
            model = model.to('cuda')
        test_loss, test_acc, test_f1, test_recall, test_kappa, test_auc = val_step(model, test_loader, train_loader, loss, device = 'cuda', classification = 'MultiClass')
        val_loss, val_acc, val_f1, val_recall, val_kappa, val_auc = val_step(model, val_loader, train_loader, loss, device = 'cuda', classification = 'MultiClass')

        val_results.append({'Model Name': idx,
                                        'Val Accuracy': val_acc,
                                        'Val F1': val_f1,
                                        'Val Recall': val_recall,
                                        'Val Kappa': val_kappa,
                                        'Val AUC': val_auc,})
        test_results.append({'Model Name': idx,
                                        'Test Accuracy': test_acc,
                                        'Test F1': test_f1,
                                        'Test Recall': test_recall,
                                        'Test Kappa': test_kappa,
                                        'Test AUC': test_auc,})
        

results_test_df = pd.DataFrame(test_results)
results_val_df = pd.DataFrame(val_results)
print(results_val_df)
print(results_test_df)



print("Uniform souping ...")
alphal = [1 / len(state_dicts) for i in range(len(state_dicts))]
model = get_model('DeiT-S', num_classes=10)
uniform_model = souping(model, state_dicts, alphal)
uniform_model.to('cuda')

uniform_test_loss, uniform_test_acc, uniform_test_f1, uniform_test_recall, uniform_test_kappa, uniform_test_auc = val_step(uniform_model, test_loader, train_loader, loss, 'cuda', 'MultiClass')

#greedy

print("Greedy souping ...")
model = get_model('DeiT-S', num_classes=10)

val_results = list(results_val_df['Val Accuracy'])
val_models = list(results_val_df["Model Name"])
greedy_model, best_ingredients = greedy_souping(state_dicts, val_results, 'DeiT-S', 10, val_loader,train_loader, loss, 'cuda' , 'MultiClass', 'Val Accuracy', val_models =val_models)
greedy_model.to('cuda')
print('VAL INGREDIENTS',best_ingredients)
greedy_test_loss, greedy_test_acc, greedy_test_f1, greedy_test_recall, greedy_test_kappa, greedy_test_auc = val_step(greedy_model, test_loader, train_loader, loss, 'cuda', 'MultiClass')
test_results = list(results_test_df['Test Accuracy'])
test_models = list(results_test_df["Model Name"])
greedy_model_test, best_ingredients_test = greedy_souping(state_dicts, test_results, 'DeiT-S', 10, test_loader, train_loader, loss, 'cuda', 'MultiClass', 'Test Accuracy', test_models)
print('TEST INGREDIENTS', best_ingredients_test)
greedy_model_test.to('cuda')
greedy_test_loss_test, greedy_test_acc_test, greedy_test_f1_test, greedy_test_recall_test, greedy_test_kappa_test, greedy_test_auc_test = val_step(greedy_model_test, test_loader, train_loader, loss, 'cuda', 'MultiClass')


['./dataset/cifar', './dataset/cifar/cifar1098_idxs.npy']
Cifar10 45000 5000 10000
1.00e-04
/l/users/santosh.sanjeev/model_soups/runs/test/fge/Cifar10_4/DeiT-S/1.00e-04/fge-0.pt


100%|██████████| 40/40 [00:23<00:00,  1.70it/s]


/l/users/santosh.sanjeev/model_soups/runs/test/fge/Cifar10_4/DeiT-S/1.00e-04/fge-1.pt


100%|██████████| 40/40 [00:23<00:00,  1.68it/s]


/l/users/santosh.sanjeev/model_soups/runs/test/fge/Cifar10_4/DeiT-S/1.00e-04/fge-13.pt


...

100%|██████████| 79/79 [00:45<00:00,  1.72it/s]


Models [0, 2, 1, 4, 7] got 0.9862539556962026 on validation.


100%|██████████| 79/79 [00:46<00:00,  1.71it/s]


Models [0, 2, 1, 4, 6] got 0.9866495253164557 on validation.
TEST INGREDIENTS [0, 2, 1, 4]


100%|██████████| 79/79 [00:45<00:00,  1.72it/s]


In [20]:
print(uniform_test_acc, greedy_test_acc, greedy_test_acc_test)

0.9866495253164557 0.9861550632911392 0.9873417721518988


In [11]:

# results_test_df = pd.DataFrame(test_results)
# results_val_df = pd.DataFrame(val_results)




In [12]:
results_val_df

,Model Name,Val Accuracy,Val F1,Val Recall,Val Kappa,Val AUC
0,0,0.989844,0.989593,0.9896,0.986788,0
1,1,0.988086,0.987784,0.9878,0.985160,0
2,2,0.985156,0.984767,0.9848,0.981920,0
3,3,0.981055,0.983561,0.9836,0.981626,0
4,4,0.983789,0.983351,0.9834,0.980323,0
5,5,0.980664,0.983167,0.9832,0.980289,0
6,6,0.979492,0.981977,0.9820,0.978611,0
7,7,0.983984,0.983580,0.9836,0.980604,0
8,8,0.981055,0.983580,0.9836,0.983637,0


In [13]:
results_test_df

,Model Name,Test Accuracy,Test F1,Test Recall,Test Kappa,Test AUC
0,0,0.987638,0.987491,0.9875,0.984240,0
1,1,0.985957,0.985791,0.9858,0.980962,0
2,2,0.983386,0.983187,0.9832,0.978572,0
3,3,0.982002,0.981785,0.9818,0.980448,0
4,4,0.983188,0.982980,0.9830,0.978452,0
5,5,0.982298,0.982087,0.9821,0.980993,0
6,6,0.982298,0.982075,0.9821,0.977957,0
7,7,0.982694,0.982496,0.9825,0.978417,0
8,8,0.980617,0.980403,0.9804,0.975170,0


In [14]:
print("Uniform souping ...")
alphal = [1 / len(state_dicts) for i in range(len(state_dicts))]
model = get_model('DeiT-S', num_classes=10)
uniform_model = souping(model, state_dicts, alphal)
uniform_model.to('cuda')

uniform_test_loss, uniform_test_acc, uniform_test_f1, uniform_test_recall, uniform_test_kappa, uniform_test_auc = val_step(uniform_model, test_loader, train_loader, loss, 'cuda', 'MultiClass')

#greedy

print("Greedy souping ...")
model = get_model('DeiT-S', num_classes=10)

val_results = list(results_val_df['Val Accuracy'])
val_models = list(results_val_df["Model Name"])
greedy_model, best_ingredients = greedy_souping(state_dicts, val_results, 'DeiT-S', 10, val_loader,train_loader, loss, 'cuda' , 'MultiClass', 'Val Accuracy', val_models =val_models)
greedy_model.to('cuda')
print('VAL INGREDIENTS',best_ingredients)
greedy_test_loss, greedy_test_acc, greedy_test_f1, greedy_test_recall, greedy_test_kappa, greedy_test_auc = val_step(greedy_model, test_loader, train_loader, loss, 'cuda', 'MultiClass')
test_results = list(results_test_df['Test Accuracy'])
test_models = list(results_test_df["Model Name"])
greedy_model_test, best_ingredients_test = greedy_souping(state_dicts, test_results, 'DeiT-S', 10, test_loader, train_loader, loss, 'cuda', 'MultiClass', 'Test Accuracy', test_models)
print('TEST INGREDIENTS', best_ingredients_test)
greedy_model_test.to('cuda')
greedy_test_loss_test, greedy_test_acc_test, greedy_test_f1_test, greedy_test_recall_test, greedy_test_kappa_test, greedy_test_auc_test = val_step(greedy_model_test, test_loader, train_loader, loss, 'cuda', 'MultiClass')


Uniform souping ...


100%|██████████| 79/79 [00:44<00:00,  1.79it/s]


Greedy souping ...
[0, 1, 2, 7, 4, 3, 8, 5, 6]
currentttttttt bestttttttt 0.98984375


100%|██████████| 40/40 [00:24<00:00,  1.62it/s]


Models [0, 1] got 0.9935546875 on validation.


...

100%|██████████| 79/79 [00:46<00:00,  1.71it/s]


Models [0, 1, 4, 3] got 0.9879351265822784 on validation.


100%|██████████| 79/79 [00:46<00:00,  1.71it/s]


Models [0, 1, 4, 8] got 0.9885284810126582 on validation.
TEST INGREDIENTS [0, 1, 4]


100%|██████████| 79/79 [00:44<00:00,  1.79it/s]


In [15]:
uniform_test_acc

0.9884295886075949

In [16]:
greedy_test_acc

0.9886273734177216

In [17]:
greedy_test_acc_test

0.9889240506329114